In [25]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape
import itertools
import os.path
import time
import os

In [26]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
product = 'MOD11A2'
modis_product = 'MODIS/006/' + product
start_year = 2011
end_year = 2011
end_year = end_year + 1 #because how range works
variables = ['QC_Day','QC_Night', 'LST_Day_1km','LST_Night_1km']
target_crs = 'SR-ORG:6974'
redl = True

In [27]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [28]:
#get a list of the city names
citynames = [item[1]['properties']['Name'] for item in the_cities]

In [29]:
#create a grid/dictionary of things to download
listolists = [variables, citynames, range(start_year,end_year)]
combos = list(itertools.product(*listolists))

In [30]:
#find which files are already downloaded
dlpath = '/modis/'
filenames = [dlpath + c[1].replace(' ','') + "_" + product +'_'+c[0]+'_'+str(c[2]) +'.tif'  for c in combos]
#already_downloaded = [os.path.isfile(fff) for fff in filenames]

In [31]:
#Read in the most recent list of files
with open('/media/dan/googledrive/modis_filelist_27.txt', 'r') as f:
    gdrivefilelist = f.read().splitlines() 

In [32]:
check = 0
for ccc in range(0, len(combos)):
    #check to see if the file is already downloaded
    if (filenames[ccc] in gdrivefilelist) == False:
        check = check +1
print check

0


In [33]:
if redl == True:
    gdrivefilelist = [""]

In [34]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [35]:
#for each possible combo
counter = 0
for ccc in range(0, len(combos)):
    #check to see if the file is already downloaded
    if (filenames[ccc] in gdrivefilelist) == False:
        year = combos[ccc][2]
        city_name_space = combos[ccc][1]
        city_name = city_name_space.replace(' ','')
        vari = combos[ccc][0]
        
        #sort out the year
        start = str(year)+'-'+'01-01'
        end = str(year)+'-'+'12-31'
        dat = imgs.filterDate(start, end).select(vari).toList(1000)

        #find the bounding box of the city
        for cit in range(0,len(the_cities)):
            if city_name_space == the_cities[cit][1]['properties']['Name']:
                bbox = shape(the_cities[cit][1]['geometry']).bounds
            

        city_name = city_name.replace(' ','')

        #name the job
        job_id = city_name + "_" + product +'_'+vari+'_'+str(year)

        result = ee.Image.cat(dat.get(0))

        #create a polygon to clip things too
        # xMin, yMin, xMax, yMax
        target_projection = ee.Projection(target_crs)
        thecity = ee.Geometry.Rectangle(bbox) #.transform(target_projection,250)

        for i in range(1,dat.length().getInfo()):
            newimage = ee.Image.cat(dat.get(i))
            result = ee.Image.cat([result, newimage])
        
        #request to download
        #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
        task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                             description = job_id, fileNamePrefix = job_id,
                                             region = thecity.getInfo().get('coordinates'),
                                             skipEmptyTiles=False)
        #print(job_id)
        task.start()
        
        
        
        if (counter +1) % 100 == 0:
            time.sleep(80)
        
        counter = counter + 1

In [21]:
dat = imgs.filterDate(start, end)

In [22]:
info = dat.getInfo().get('features')
for i in range(0, len(info)):
    print info[i].get('id')

MODIS/006/MOD11A2/2011_01_01
MODIS/006/MOD11A2/2011_01_09
MODIS/006/MOD11A2/2011_01_17
MODIS/006/MOD11A2/2011_01_25
MODIS/006/MOD11A2/2011_02_02
MODIS/006/MOD11A2/2011_02_10
MODIS/006/MOD11A2/2011_02_18
MODIS/006/MOD11A2/2011_02_26
MODIS/006/MOD11A2/2011_03_06
MODIS/006/MOD11A2/2011_03_14
MODIS/006/MOD11A2/2011_03_22
MODIS/006/MOD11A2/2011_03_30
MODIS/006/MOD11A2/2011_04_07
MODIS/006/MOD11A2/2011_04_15
MODIS/006/MOD11A2/2011_04_23
MODIS/006/MOD11A2/2011_05_01
MODIS/006/MOD11A2/2011_05_09
MODIS/006/MOD11A2/2011_05_17
MODIS/006/MOD11A2/2011_05_25
MODIS/006/MOD11A2/2011_06_02
MODIS/006/MOD11A2/2011_06_10
MODIS/006/MOD11A2/2011_06_18
MODIS/006/MOD11A2/2011_06_26
MODIS/006/MOD11A2/2011_07_04
MODIS/006/MOD11A2/2011_07_12
MODIS/006/MOD11A2/2011_07_20
MODIS/006/MOD11A2/2011_07_28
MODIS/006/MOD11A2/2011_08_05
MODIS/006/MOD11A2/2011_08_13
MODIS/006/MOD11A2/2011_08_21
MODIS/006/MOD11A2/2011_08_29
MODIS/006/MOD11A2/2011_09_06
MODIS/006/MOD11A2/2011_09_14
MODIS/006/MOD11A2/2011_09_22
MODIS/006/MOD1